In [2]:
import os
import re
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.api_core.exceptions import Conflict

# ----------------------------
# 1. Authentication and Setup
# ----------------------------
PROJECT_ID = "even-blueprint-441418-p2"
DATASET_ID = "ecommerce_Bronze_Layer"  # Name of the dataset in BigQuery
KEY_PATH = r"C:\Users\Vishnu Vardhan\OneDrive\Desktop\Bigquery_Ecommerce\even-blueprint-441418-p2-043f8a9d855b.json"
folder = r"C:\Users\Vishnu Vardhan\OneDrive\Desktop\Bigquery_Ecommerce\Table_Data_Files"

# Authenticate using the service account key
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)
# Merge all CSVs
dfs = []
for file in os.listdir(folder):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(folder, file))
        df["source_table"] = file.replace(".csv", "")
        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

# Save combined CSV
combined_df.to_csv("combined.csv", index=False)

# Upload to BigQuery
table_id = "even-blueprint-441418-p2.ecommerce_Bronze_Layer.raw_Brazilian_data"
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV,
    write_disposition="WRITE_TRUNCATE"
)

with open("combined.csv", "rb") as f:
    load_job = client.load_table_from_file(f, table_id, job_config=job_config)
    load_job.result()

print("✅ All CSV files merged and uploaded into all_data table!")


✅ All CSV files merged and uploaded into all_data table!
